In [14]:
import json
import requests
import xml.etree.ElementTree as ET
import time
import pandas as pd
import seaborn as sns
from tqdm.auto import tqdm
import re
from datetime import datetime, timedelta
import os
from IPython.display import clear_output

In [15]:
base_data = json.load(open('info.json', encoding='utf-8'))
epoch_start_in_minutes = 84150720
epoch_start_date = datetime(1960, 1, 1)

In [50]:
# Function to convert a minute-based offset to a date, given an epoch
def get_days_in_month(year, month):
    start_m = month - 1 if month < 1 else 12
    start_y = year if month > 1 else year - 1
    days_in_month = (datetime(year, month, 1) - datetime(start_y, start_m, 1)).days
    return days_in_month

def convert_to_date(minutes):
    days_since_epoch = (minutes - epoch_start_in_minutes) / 1440
    target_date = epoch_start_date + timedelta(days=days_since_epoch)
    return target_date.strftime('%Y-%m-%d')

class Rudari_Glavne_Postaje:
    def __init__(self, postaja: str, start: str, stop: str = None):
        os.system("cls")
        print(f"Pričenjam z rudarjenjem podatkov za {postaja}")
        self.base_data = json.load(open('info.json', encoding='utf-8'))
        self.postaja = postaja
        self.id = f"{self.base_data[postaja]['ID'].replace('id=','_')}"
        self.df = {"param":None,"data":None}
        self.data = {}
        self.params = None
        self.str_to_date = lambda x: datetime.strptime(x, '%Y-%m-%d')
        self.check_start = self.str_to_date(self.base_data[postaja]["start"])
        self.start = self.str_to_date(start)
        
        if self.start < self.check_start:
            self.start = self.check_start
            print(f"Start date is before the first available date. Setting start date to {self.start}")
            
        if stop != None:
            self.stop = self.str_to_date(stop)
        else:
            self.stop = None
        self.start_minning()
        
    def construct_url(self, postaja, od, do):
        #print("Constructing URL")
        id = self.base_data[postaja]["ID"]
        return self.base_data["baseurl"] + f"{id}&d1={od}&d2={do}"

    def parameter_names(self):
        #print("Getting parameter names")
        p = self.data.get("o", [])
        return {key: self.data["params"][key]["l"] for key in p}

    def parse_content(self, content: str):
        #print("Parsing content")
        root = ET.fromstring(content)
        cdata_content = root.text
        js_object_str = re.search(r'{.*}', cdata_content, re.DOTALL).group()
        js_object_str = re.sub(r'(?<!")(\b\w+\b)(?!"):', r'"\1":', js_object_str)
        try:
            return json.loads(js_object_str)
        except json.JSONDecodeError as e:
            print("Error parsing JSON:", e)
            return {}

    def parse_xml_from_link(self, link):
        #print("Parsing XML from link")
        response = requests.get(link)
        if response.encoding is not None:
            content = response.content.decode('utf-8', errors='replace')
        return self.parse_content(content)
    
    def start_minning(self):
        init_year = self.check_start.year
        init_month = self.check_start.month
        
        if self.stop != None:
            final_year = self.stop.year
            final_month = self.stop.month
        else:
            final_year = datetime.now().year
            final_month = datetime.now().month
        
        year = self.start.year
        for year in tqdm(range(self.start.year,final_year +1, 1),total = final_year - year + 1, desc = "Leto"):
            yf = year
            day_s = 1
            day_f = 31
            mf = 12
            if yf > final_year:
                yf = final_year
                month = final_month-1
                mf = month + 1
                day_f =  get_days_in_month(yf, month)
                    
            od = f"{year}-01-{day_s:02d}"
            do = f"{yf}-{mf:02d}-{day_f:02d}"
            url = self.construct_url(self.postaja, od, do)
            # clear_output(wait=True)
            # print(f"    - {self.postaja}: {od} -> {do} ",url)
            if url is not None:
                self.data = self.parse_xml_from_link(url)
                meritve = self.data["points"][self.id]
                dan = len(meritve)
                #print(dan,meritve)
                self.params = self.parameter_names()
                if self.df["data"]==None:
                    columns = ["Datum"] + list(self.params.values())
                    self.df["data"] = {col:[] for col in columns}
                    self.df["param"] = list(self.params.values())
                
                for i,key1 in enumerate(meritve):
                    #print(key1)
                    unix_time = convert_to_date(int(key1[1::]))
                    #print(key1,int(key1[1::]),unix_time)
                    self.df["data"]["Datum"].append(unix_time)
                    for key2 in self.params:
                        value = meritve[key1].get(key2 , None)
                        key = self.params[key2]
                        self.df["data"][key].append(value)
                            #print(datum,key,key2,value)
            #os.system("cls")
            #input("Nadaljuje")
        
        path = os.path.join("zbrani_podatki","json",f"{self.postaja}.json")
        with open(path, "w",encoding="utf-8") as f:
            json.dump(self.df, f, default=str, indent=4, ensure_ascii=False)
        clear_output(wait=True)
        #print(f"Zadnji zajet datum za {self.postaja} je {do}")
        return None

class Analiziriraj_postajo:
    def __init__(self, postaja: str):
        self.postaja = postaja
        self.data = json.load(open(os.path.join("zbrani_podatki","json",f"{postaja}.json"), encoding='utf-8'))
        self.df = pd.DataFrame(self.data["data"])
        self.df["Datum"] = [datetime.strptime(x, '%d-%m-%Y') for x in self.df["Datum"]]
    
    def analiza_padavin(self):
        
        print("Analiziram padavine")
        padavine = self.df["24-urna količina padavin ob 7 h (mm)"]
        


In [51]:
for i,ii in enumerate(base_data):
    if i>0:
        print(i,ii)

1 AJDOVŠČINA
2 Babno Polje
3 Bilje
4 Bizeljsko
5 Bohinjska Češnjica
6 CELJE - LOKROVEC
7 Črnomelj - Dobliče
8 CELJE - ŽALEC
9 KOPER
10 Kredarica
11 Lisca
12 LJUBLJANA - AERODROM
13 Ljubljana Bežigrad
14 Maribor - letališče
15 MARIBOR - TEZNO
16 MURSKA SOBOTA - RAKIČAN
17 NOVO MESTO - GOTNA VAS
18 NOVO MESTO - KANDIJA
19 POSTOJNA
20 RATEČE
21 ŠMARTNO PRI SLOV. GRADCU
22 ZGORNJE JEZERSKO
23 GODNJE
24 Grosuplje
25 IVANJKOVCI
26 Kočevje
27 Lendava
28 METLIKA
29 Nova vas (Bloke)
30 Polički Vrh
31 SLOVENSKE KONJICE
32 STARŠE
33 Vojsko (Idrija)


In [52]:
rudari = Rudari_Glavne_Postaje("AJDOVŠČINA","1960-01-01")

Pričenjam z rudarjenjem podatkov za AJDOVŠČINA


Leto:   0%|          | 0/65 [00:00<?, ?it/s]

In [53]:
def quality_check(postaja):
    data = json.load(open(os.path.join("zbrani_podatki","json",f"{postaja}.json"), encoding='utf-8'))
    df = pd.DataFrame(data["data"])
    N0 = len(df)
    #Drop all data with None values
    #df = df.dropna(axis=0, how='any')
    df = df.dropna(subset=['povprečna temperatura zraka na 2 m (°C)'])
    N1 = len(df)
    print(f"Pregled za postajo {postaja}:")
    print(" - Prvi zabeležen dan:",df["Datum"].min())
    print(" - Zadnji zabeležen dan:",df["Datum"].max())
    print(" - Število vrstic pred čiščenjem:",N0)
    print(" - Število vrstic po čiščenju:",N1)
    print(f" - Odstranjeno je bilo {((N0-N1)/N0*100.0):.2f}% vrstic z manjkajočimi podatki")
    df.to_excel(os.path.join("zbrani_podatki","excel",f"{postaja}.xlsx"),index=False)
    print()

In [54]:
pridobi = False
for i,merilna_postaja in enumerate(base_data):
    if i>0:
        if merilna_postaja=="METLIKA":
            pridobi = True
        rudari = Rudari_Glavne_Postaje(merilna_postaja,"1961-01-01")
        quality_check(merilna_postaja)

Pregled za postajo Lendava:
 - Prvi zabeležen dan: 1962-01-17
 - Zadnji zabeležen dan: 2024-02-29
 - Število vrstic pred čiščenjem: 23376
 - Število vrstic po čiščenju: 22687
 - Odstranjeno je bilo 2.95% vrstic z manjkajočimi podatki


SerialisationError: IO_WRITE

In [41]:
for i,merilna_postaja in enumerate(rudari.base_data):
    if i>0:
        print(merilna_postaja)
        quality_check(merilna_postaja)
        print()

AJDOVŠČINA
Odstranjeno je bilo 100.00% vrstic z manjkajočimi podatki

CELJE - LOKROVEC
Odstranjeno je bilo 100.00% vrstic z manjkajočimi podatki

CELJE - ŽALEC
Odstranjeno je bilo 100.00% vrstic z manjkajočimi podatki

KOPER
Odstranjeno je bilo 78.88% vrstic z manjkajočimi podatki

Kredarica
Odstranjeno je bilo 100.00% vrstic z manjkajočimi podatki

LJUBLJANA - AERODROM
Odstranjeno je bilo 100.00% vrstic z manjkajočimi podatki

Ljubljana Bežigrad
Odstranjeno je bilo 0.10% vrstic z manjkajočimi podatki

MARIBOR - TEZNO
Odstranjeno je bilo 97.73% vrstic z manjkajočimi podatki

MURSKA SOBOTA - RAKIČAN
Odstranjeno je bilo 83.81% vrstic z manjkajočimi podatki

NOVO MESTO - GOTNA VAS
Odstranjeno je bilo 81.49% vrstic z manjkajočimi podatki

NOVO MESTO - KANDIJA
Odstranjeno je bilo 100.00% vrstic z manjkajočimi podatki

POSTOJNA
Odstranjeno je bilo 99.63% vrstic z manjkajočimi podatki

RATEČE
Odstranjeno je bilo 28.47% vrstic z manjkajočimi podatki

ŠMARTNO PRI SLOV. GRADCU
Odstranjeno je bil

In [5]:

link = construct_url("MARIBOR - TEZNO",start = "1961-01-01",stop = "1961-02-01")
print(link)
response = requests.get(link)
if response.encoding is not None:
    content = response.content.decode('utf-8', errors='replace')
data = parse_content(content)
map_parameter_names = parameter_names(data)

id=1061 1961-01-01 1961-02-01
https://meteo.arso.gov.si/webmet/archive/data.xml?lang=si&vars=35,56,38,57,36,58,37,59,43,60,46,61,40,62,33,63,85,65,88,66,89,67,41,68,80,69,81,70,47,71,48,72,49,74,50,75,51,77,52,78,53,83,54,82,55&group=dailyData2&type=daily&id=1061&d1=1961-01-01&d2=1961-02-01
{ "baseurl":"./icons/", "gen":"ACADEMA.AEP.getGenerators('archive').genTerminData('yyyy-MM-dd')", "datatype":"daily", "o":[ "p0" , "p1" , "p2" , "p3" , "p4" , "p5" , "p6" , "p7" , "p8" , "p9" , "p10" , "p11" , "p12" , "p13" , "p14" , "p15" , "p16" , "p17" , "p18" , "p19" , "p20" , "p21" , "p22" , "p23" , "p24" , "p25" , "p26" , "p27" , "p28" , "p29" , "p30" , "p31" , "p32" , "p33" , "p34" , "p35" , "p36" , "p37" , "p38" , "p39" , "p40" , "p41" , "p42" , "p43" , "p44" ], "params":{"p0":{ "pid":"35", "name":"t2m_klima", "s":"povp. dnevna T", "l":"povprečna temperatura zraka na 2 m (°C)", "unit":"°C"},"p1":{ "pid":"38", "name":"tmax", "s":"max. T", "l":"maksimalna temperatura zraka na 2 m (°C)", "unit"

In [9]:
param

{'p0': 'povprečna temperatura zraka na 2 m (°C)',
 'p1': 'maksimalna temperatura zraka na 2 m (°C)',
 'p2': 'minimalna temperatura zraka na 2 m (°C)',
 'p3': 'minimalna temperatura zraka na 5cm (°C)',
 'p4': 'povprečna hitrost vetra (m/s)',
 'p5': 'povprečna oblačnost (pokritost neba v %)',
 'p6': 'povprečna relativna vlaga (%)',
 'p7': 'povprečen zračni tlak (hPa)',
 'p8': '24-urna količina padavin ob 7 h (mm)',
 'p9': 'skupna višina snežne odeje (cm)',
 'p10': 'višina novozapadlega snega (cm)',
 'p11': 'trajanje sončnega obsevanja (h)',
 'p12': 'močan veter (> 6bf)',
 'p13': 'viharni veter (> 8bf)',
 'p14': 'dež',
 'p15': 'rosenje',
 'p16': 'ploha dežja',
 'p17': 'nevihta',
 'p18': 'grmenje',
 'p19': 'bliskanje',
 'p20': 'dež, ki zmrzuje',
 'p21': 'rosenje, ki zmrzuje',
 'p22': 'ledene iglice',
 'p23': 'sneg',
 'p24': 'zrnat sneg',
 'p25': 'ploha snega',
 'p26': 'dež s snegom',
 'p27': 'babje pšeno',
 'p28': 'ploha dežja s snegom',
 'p29': 'toča',
 'p30': 'sodra',
 'p31': 'megla',
 '